In [1]:
import gevent
import pandas as pd
import numpy as np
import math
import pymysql
import time 
import functools as ft
import glob, os   
import operator as op
import shelve
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from pandas.api.types import is_numeric_dtype
from pathlib import Path
from itertools import combinations, product, permutations
from sqlalchemy.engine import create_engine
from datetime import datetime
from ast import literal_eval
from scipy import stats  
from scipy.stats.mstats import gmean
from pythonds.basic.stack import Stack
from pythonds.trees.binaryTree import BinaryTree
from collections import defaultdict
import collections
from typing import List, Set, Tuple 


data_directory = '/Users/gms/development/nlp/nlpie/data/ensembling-u01/output/' 

In [ ]:
# RESULTS

def get_results():
    results = pd.DataFrame()
    for fname in glob.glob(data_directory + '/submission/combined/*.csv'):

        #print(fname)
        t = os.path.basename(fname)
        corpus = t.split('_')[0]
        #print(corpus)
        semtypes = t.split('_')[-2:-1][0]
        print(t.split('_')[-2:-1], t, corpus, semtypes)

        temp = pd.read_csv(fname) 

        temp['corpus'] = corpus
        temp['semtypes'] = semtypes
        temp['file'] = t

        frames = [ temp, results ]
        results = pd.concat(frames)


    merges = results.copy()
    merges = merges.rename(index=str, columns={"n_gold": "n_ref"})

    # sfingle system evaluation
    cols_to_keep = ['merge', 'corpus', 'semtypes', 'F', 'precision', 'recall', 'n_sys', 'n_gold']    
    #print(results[cols_to_keep][results['merge'].isin(['biomedicus', 'clamp', 'ctakes', 'metamap', 'quick_umls'])].sort_values(by=['corpus', 'merge', 'semtypes']).rename(index=str, columns={"merge": "system", "n_gold": "n_ref"}))
    df = results[cols_to_keep][results['merge'].isin(['biomedicus', 'clamp', 'ctakes', 'metamap', 'quick_umls'])].sort_values(by=['corpus', 'merge', 'semtypes']).rename(index=str, columns={"merge": "system", "n_gold": "n_ref"}).copy()
    #df.to_csv(data_directory + '/submission/single_system_summary.csv')

    merges.reset_index(inplace=True)
    merges = merges[~merges['merge'].isin(['biomedicus', 'clamp', 'ctakes', 'metamap', 'quick_umls'])]
    max_f = merges.copy()
    max_p = merges.copy()
    max_r = merges.copy()
    #print(len(max_f), len(max_p), len(max_r))
    
    # https://datascience.stackexchange.com/questions/26308/after-grouping-to-minimum-value-in-pandas-how-to-display-the-matching-row-resul
    cols_to_keep = ['merge', 'corpus', 'semtypes', 'F', 'precision', 'recall', 'n_sys', 'n_ref']    
    f = max_f.loc[max_f.groupby(['corpus','semtypes'])['F'].idxmax()]
    f = f[cols_to_keep].sort_values(by=['corpus','semtypes'])

    cols_to_keep = ['merge', 'corpus', 'semtypes', 'F', 'precision', 'recall', 'n_sys', 'n_ref']    
    p = max_p.loc[max_p.groupby(['corpus','semtypes'])['precision'].idxmax()]
    p = p[cols_to_keep].sort_values(by=['corpus','semtypes'])

    cols_to_keep = ['merge', 'corpus', 'semtypes', 'F', 'precision', 'recall', 'n_sys', 'n_ref']    
    r = max_r.loc[max_r.groupby(['corpus','semtypes'])['recall'].idxmax()]
    r = r[cols_to_keep].sort_values(by=['corpus','semtypes'])
    #print(len(f), len(p), len(r))
    #print(f, p, r)

    writer = pd.ExcelWriter(data_directory + '/submission/max_merge_summary.xlsx', engine='xlsxwriter')
    f.to_excel(writer, sheet_name='max F-score')
    p.to_excel(writer, sheet_name='max precision')
    r.to_excel(writer, sheet_name='max recall')
    writer.save()
    
get_results()

In [20]:
import pandas as pd
from scipy import stats
from scipy.stats.mstats import gmean

def geometric_mean(metrics):
    """
    1. Get rank average of F1, TP/FN, TM
        http://www.datasciencemadesimple.com/rank-dataframe-python-pandas-min-max-dense-rank-group/
        https://stackoverflow.com/questions/46686315/in-pandas-how-to-create-a-new-column-with-a-rank-according-to-the-mean-values-o?rq=1
    2. Take geomean of 2.
        https://stackoverflow.com/questions/42436577/geometric-mean-applied-on-row
    """
    
    data = pd.DataFrame() 

    metrics['F1 rank']=metrics['F'].rank(ascending=0,method='average')
    metrics['TP/FN rank']=metrics['TP/FN'].rank(ascending=0,method='average')
    metrics['TM rank']=metrics['TM'].rank(ascending=0,method='average')
    metrics['Gmean'] = gmean(metrics.iloc[:,-3:],axis=1)

    return metrics  

In [26]:
def get_vote():
    results = pd.DataFrame()
    for fname in glob.glob(data_directory + '/submission/vote/*.csv'):

        #print(fname)
        t = os.path.basename(fname)
        corpus = t.split('_')[0]
        print(corpus)
        semtypes = t.split('_')[-2:-1][0]
        print(t.split('_')[-2:-1], t, corpus, semtypes)

        temp = pd.read_csv(fname) 

        temp['corpus'] = corpus
        temp['semtypes'] = semtypes
        temp['file'] = t

        frames = [ temp, results ]
        results = pd.concat(frames)
        
    merges = results.copy()
    merges = merges.rename(index=str, columns={"n_gold": "n_ref"})
    
    # sfingle system evaluation
    cols_to_keep = ['merge', 'corpus', 'semtypes', 'F', 'precision', 'recall', 'n_sys', 'n_gold']    
    #print(results[cols_to_keep][results['merge'].isin(['biomedicus', 'clamp', 'ctakes', 'metamap', 'quick_umls'])].sort_values(by=['corpus', 'merge', 'semtypes']).rename(index=str, columns={"merge": "system", "n_gold": "n_ref"}))
    #df = results[cols_to_keep][results['merge'].isin(['biomedicus', 'clamp', 'ctakes', 'metamap', 'quick_umls'])].sort_values(by=['corpus', 'merge', 'semtypes']).rename(index=str, columns={"merge": "system", "n_gold": "n_ref"}).copy()
    results.to_csv(data_directory + '/submission/vote_summary.csv')

#get_vote()
def update_complete_data():
    metrics = pd.DataFrame()
    for fname in glob.glob(data_directory + '/submission/combined/*.csv'):

        #print(fname)
        t = os.path.basename(fname)
        u = t.split('.')[0]
        print(u)
        now = datetime.now()
        timestamp = datetime.timestamp(now)
        
        df = pd.read_csv(fname)
        df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'F1 rank', 'TP/FN rank', 'TM rank', 'Gmean'], axis=1, inplace=True)
        #print(df.head(1))
        #print(geometric_mean(df)) 
        
        new_file_name = u + '_' + str(timestamp) + '.csv'
        geometric_mean(df).to_csv(data_directory + '/submission/combined/' + new_file_name)
        
update_complete_data()

mipacq_Anatomy_combined
mipacq_Disorders,Sign-Symptom_combined
mipacq_Procedures_combined
i2b2_test,treatment_combined
mipacq_Chemicals-drug_combined
i2b2_problem_combined
mipacq_all_combined
fairview_Drug_combined
fairview_all_combined
fairview_Procedure_combined
fairview_Finding_combined
i2b2_all_combined
fairview_Anatomy_combined


In [ ]:
'''
df = pd.read_csv(data_directory + '/submission/single_system_summary.csv')
#print(df)

out = pd.pivot_table(df, values = ['precision', 'recall', 'F'], index=['corpus','semtypes'], columns = 'system').reset_index()

out.to_csv(data_directory + '/submission/single_system_out.csv')
'''
#df = pd.read_csv(data_directory + '/submission/test.csv')
# #print(df)

#pd.pivot_table(df, values = ['clinical_m1', 'clinical_m2'], index=['test_number','clinical_type'], columns = 'system').reset_index()

In [ ]:
'''
from functools import reduce
f = pd.read_excel(open(data_directory + '/submission/max_merge_summary.xlsx', 'rb'), sheet_name='max F-score')
#print(f)

p = pd.read_excel(open(data_directory + '/submission/max_merge_summary.xlsx', 'rb'), sheet_name='max precision')
#print(p)

r = pd.read_excel(open(data_directory + '/submission/max_merge_summary.xlsx', 'rb'), sheet_name='max recall')
#print(r)

data_frames = [f, p, r]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['corpus', 'semtypes'], how='inner'), data_frames)
df_merged = df_merged.rename(index=str, columns={"merge_x": "merge_f", "merge_y": "merge_p", "merge": 'merge_r', 'n_ref_x': 'n_ref'})

#print(df_merged)
cols_to_keep = ['merge_f', 'merge_p', 'merge_r', 'F', 'precision', 'recall', 'n_ref', 'corpus', 'semtypes']
print(df_merged[cols_to_keep])

df_merged[cols_to_keep].to_csv(data_directory + '/submission/max_merge_out.csv')
'''

In [ ]:
'''
nested:
fairview_all_merge_metrics_entity_overlap_Anatomy_1578694661.483111.csv
fairview_all_merge_metrics_entity_overlap_Drug_1578691637.613012.csv
fairview_all_merge_metrics_entity_overlap_Finding_1578694540.707012.csv
fairview_all_merge_metrics_entity_overlap_Procedure_1578695994.216092.csv
fairview_all_merge_metrics_entity_overlap_complete_1578628666.2874.csv
i2b2_all_merge_metrics_entity_overlap_complete_1578635243.689393.csv
i2b2_all_merge_metrics_entity_overlap_problem_1578689510.705959.csv
i2b2_all_merge_metrics_entity_overlap_test,treatment_1578686098.336592.csv
mipacq_all_merge_metrics_entity_overlap_Anatomy_1578680047.974938.csv
mipacq_all_merge_metrics_entity_overlap_Chemicals_and_drugs_1578682645.26684.csv
mipacq_all_merge_metrics_entity_overlap_Disorders,Sign_Symptom_1578679769.464332.csv
mipacq_all_merge_metrics_entity_overlap_Procedures_1578676610.30274.csv
mipacq_all_merge_metrics_entity_overlap_complete_1578672135.059832.csv
'''

'''
nested_with_singleton:
fairview_all_merge_metrics_entity_overlap_1578945459.512492.csv
fairview_all_merge_metrics_entity_overlap_Drug1578942397.436205.csv
fairview_all_merge_metrics_entity_overlap_Finding1578942519.761993.csv
fairview_all_merge_metrics_entity_overlap_Procedure1578942581.064223.csv
i2b2_all_merge_metrics_entity_overlap_1578945078.21741.csv
i2b2_all_merge_metrics_entity_overlap_problem1578943079.274995.csv
i2b2_all_merge_metrics_entity_overlap_test,treatment1578942903.11009.csv
mipacq_all_merge_metrics_entity_overlap_1578944504.010892.csv
mipacq_all_merge_metrics_entity_overlap_Chemicals_and_drugs1578944172.737039.csv
mipacq_all_merge_metrics_entity_overlap_Disorders,Sign_Symptom1578944028.8403.csv
mipacq_all_merge_metrics_entity_overlap_Procedures1578943851.142459.csv
'''

'''
paired:
fairview_all_merge_metrics_entity_overlap_1578946718.235075.csv
fairview_all_merge_metrics_entity_overlap_Anatomy1578953589.256364.csv
fairview_all_merge_metrics_entity_overlap_Drug1578953297.264332.csv
fairview_all_merge_metrics_entity_overlap_Finding1578953578.940667.csv
fairview_all_merge_metrics_entity_overlap_Procedure1578953730.388245.csv
i2b2_all_merge_metrics_entity_overlap_1578949083.34194.csv
i2b2_all_merge_metrics_entity_overlap_problem1578951222.071805.csv
i2b2_all_merge_metrics_entity_overlap_test,treatment1578950806.296343.csv
mipacq_all_merge_metrics_entity_overlap_1578948214.410456.csv
mipacq_all_merge_metrics_entity_overlap_Anatomy1578952051.5378.csv
mipacq_all_merge_metrics_entity_overlap_Chemicals_and_drugs1578952386.368777.csv
mipacq_all_merge_metrics_entity_overlap_Disorders,Sign_Symptom1578952028.443752.csv
mipacq_all_merge_metrics_entity_overlap_Procedures1578951619.047393.csv
'''

In [ ]:
'''
# combine data sets

# fv
n = pd.read_csv(data_directory + '/submission/fairview_all_merge_metrics_entity_overlap_Anatomy_1578694661.483111.csv')
p = pd.read_csv(data_directory + '/submission/paired/fairview_all_merge_metrics_entity_overlap_Anatomy1578953589.256364.csv')

out = pd.concat([n, p])

out.to_csv(data_directory + '/submission/combined/fairview_Anatomy.csv')
          
print('fv anat', len(out), out.sort_values(by='F',ascending=False)[['F', 'merge']].head(1).values[0], out.sort_values(by='precision',ascending=False)[['precision', 'merge']].head(1).values[0], 
      out.sort_values(by='recall',ascending=False)[['recall', 'merge']].head(1).values[0])

n = pd.read_csv(data_directory + '/submission/fairview_all_merge_metrics_entity_overlap_Drug_1578691637.613012.csv')
ns = pd.read_csv(data_directory + '/submission/nested_with_singleton/fairview_all_merge_metrics_entity_overlap_Drug1578942397.436205.csv')
p = pd.read_csv(data_directory + '/submission/paired/fairview_all_merge_metrics_entity_overlap_Drug1578953297.264332.csv')

out = pd.concat([n, ns, p])

out.to_csv(data_directory + '/submission/combined/fairview_Drug.csv')

print('fv drug', len(out), out.sort_values(by='F',ascending=False)[['F', 'merge']].head(1).values[0], out.sort_values(by='precision',ascending=False)[['precision', 'merge']].head(1).values[0], 
      out.sort_values(by='recall',ascending=False)[['recall', 'merge']].head(1).values[0])

n = pd.read_csv(data_directory + '/submission/fairview_all_merge_metrics_entity_overlap_Finding_1578694540.707012.csv')
ns = pd.read_csv(data_directory + '/submission/nested_with_singleton/fairview_all_merge_metrics_entity_overlap_Finding1578942519.761993.csv')
p = pd.read_csv(data_directory + '/submission/paired/fairview_all_merge_metrics_entity_overlap_Finding1578953578.940667.csv')

out = pd.concat([n, ns, p])

out.to_csv(data_directory + '/submission/combined/fairview_Finding.csv')

print('fv find', len(out), out.sort_values(by='F',ascending=False)[['F', 'merge']].head(1).values[0], out.sort_values(by='precision',ascending=False)[['precision', 'merge']].head(1).values[0], 
      out.sort_values(by='recall',ascending=False)[['recall', 'merge']].head(1).values[0])

n = pd.read_csv(data_directory + '/submission/fairview_all_merge_metrics_entity_overlap_Procedure_1578695994.216092.csv')
ns = pd.read_csv(data_directory + '/submission/nested_with_singleton/fairview_all_merge_metrics_entity_overlap_Procedure1578942581.064223.csv')
p = pd.read_csv(data_directory + '/submission/paired/fairview_all_merge_metrics_entity_overlap_Procedure1578953730.388245.csv')

out = pd.concat([n, ns, p])

out.to_csv(data_directory + '/submission/combined/fairview_Procedure.csv')

print('fv proc', len(out), out.sort_values(by='F',ascending=False)[['F', 'merge']].head(1).values[0], out.sort_values(by='precision',ascending=False)[['precision', 'merge']].head(1).values[0], 
      out.sort_values(by='recall',ascending=False)[['recall', 'merge']].head(1).values[0])

n = pd.read_csv(data_directory + '/submission/fairview_all_merge_metrics_entity_overlap_complete_1578628666.2874.csv')
ns = pd.read_csv(data_directory + '/submission/nested_with_singleton/fairview_all_merge_metrics_entity_overlap_1578945459.512492.csv')
p = pd.read_csv(data_directory + '/submission/paired/fairview_all_merge_metrics_entity_overlap_1578946718.235075.csv')

out = pd.concat([n, ns, p])

out.to_csv(data_directory + '/submission/combined/fairview_complete.csv')

print('fv all', len(out), out.sort_values(by='F',ascending=False)[['F', 'merge']].head(1).values[0], out.sort_values(by='precision',ascending=False)[['precision', 'merge']].head(1).values[0], 
      out.sort_values(by='recall',ascending=False)[['recall', 'merge']].head(1).values[0])

# i2b2


n = pd.read_csv(data_directory + '/submission/i2b2_all_merge_metrics_entity_overlap_test,treatment_1578686098.336592.csv')
ns = pd.read_csv(data_directory + '/submission/nested_with_singleton/i2b2_all_merge_metrics_entity_overlap_test,treatment1578942903.11009.csv')
p = pd.read_csv(data_directory + '/submission/paired/i2b2_all_merge_metrics_entity_overlap_test,treatment1578950806.296343.csv')

out = pd.concat([n, ns, p])

out.to_csv(data_directory + '/submission/combined/i2b2_test,treatment.csv')

print('i2b2 proc', len(out), out.sort_values(by='F',ascending=False)[['F', 'merge']].head(1), out.sort_values(by='precision',ascending=False)[['precision', 'merge']].head(1), out.sort_values(by='recall',ascending=False)[['recall', 'merge']].head(1)) 

n = pd.read_csv(data_directory + '/submission/i2b2_all_merge_metrics_entity_overlap_problem_1578689510.705959.csv')
ns = pd.read_csv(data_directory + '/submission/nested_with_singleton/i2b2_all_merge_metrics_entity_overlap_problem1578943079.274995.csv')
p = pd.read_csv(data_directory + '/submission/paired/i2b2_all_merge_metrics_entity_overlap_problem1578951222.071805.csv')    

out = pd.concat([n, ns, p])

out.to_csv(data_directory + '/submission/combined/i2b2_problem.csv')

print('i2b2 find', len(out), out.sort_values(by='F',ascending=False)[['F', 'merge']].head(1).values[0], out.sort_values(by='precision',ascending=False)[['precision', 'merge']].head(1).values[0], 
      out.sort_values(by='recall',ascending=False)[['recall', 'merge']].head(1).values[0])

n = pd.read_csv(data_directory + '/submission/i2b2_all_merge_metrics_entity_overlap_complete_1578635243.689393.csv')
ns = pd.read_csv(data_directory + '/submission/nested_with_singleton/i2b2_all_merge_metrics_entity_overlap_1578945078.21741.csv')
p = pd.read_csv(data_directory + '/submission/paired/i2b2_all_merge_metrics_entity_overlap_1578949083.34194.csv')

out = pd.concat([n, ns, p])

out.to_csv(data_directory + '/submission/combined/i2b2_all.csv')

print('i2b2 all', out.sort_values(by='F',ascending=False)[['F', 'merge']].head(1).values[0], out.sort_values(by='precision',ascending=False)[['precision', 'merge']].head(1).values[0], 
      out.sort_values(by='recall',ascending=False)[['recall', 'merge']].head(1).values[0])

# mipacq

n = pd.read_csv(data_directory + '/submission/mipacq_all_merge_metrics_entity_overlap_Anatomy_1578680047.974938.csv')
p = pd.read_csv(data_directory + '/submission/paired/mipacq_all_merge_metrics_entity_overlap_Anatomy1578952051.5378.csv')

out = pd.concat([n, p])

out.to_csv(data_directory + '/submission/combined/mipacq_Anatomy.csv')

print('m anat', len(out), out.sort_values(by='F',ascending=False)[['F', 'merge']].head(1).values[0], out.sort_values(by='precision',ascending=False)[['precision', 'merge']].head(1).values[0], 
      out.sort_values(by='recall',ascending=False)[['recall', 'merge']].head(1).values[0])

n = pd.read_csv(data_directory + '/submission/mipacq_all_merge_metrics_entity_overlap_Chemicals_and_drugs_1578682645.26684.csv')
ns = pd.read_csv(data_directory + '/submission/nested_with_singleton/mipacq_all_merge_metrics_entity_overlap_Chemicals_and_drugs1578944172.737039.csv')
p = pd.read_csv(data_directory + '/submission/paired/mipacq_all_merge_metrics_entity_overlap_Chemicals_and_drugs1578952386.368777.csv')

out = pd.concat([n, ns, p])

out.to_csv(data_directory + '/submission/combined/mipacq_Chemicals_and_drug.csv')

print('m drug', len(out), out.sort_values(by='F',ascending=False)[['F', 'merge']].head(1).values[0], out.sort_values(by='precision',ascending=False)[['precision', 'merge']].head(1).values[0], 
      out.sort_values(by='recall',ascending=False)[['recall', 'merge']].head(1).values[0])

n = pd.read_csv(data_directory + '/submission/mipacq_all_merge_metrics_entity_overlap_Disorders,Sign_Symptom_1578679769.464332.csv')
ns = pd.read_csv(data_directory + '/submission/nested_with_singleton/mipacq_all_merge_metrics_entity_overlap_Disorders,Sign_Symptom1578944028.8403.csv')
p = pd.read_csv(data_directory + '/submission/paired/mipacq_all_merge_metrics_entity_overlap_Disorders,Sign_Symptom1578952028.443752.csv')

out = pd.concat([n, ns, p])

out.to_csv(data_directory + '/submission/combined/mipacq_Disorders,Sign_Symptom.csv')

print('m find', len(out), out.sort_values(by='F',ascending=False)[['F', 'merge']].head(1).values[0], out.sort_values(by='precision',ascending=False)[['precision', 'merge']].head(1).values[0], 
      out.sort_values(by='recall',ascending=False)[['recall', 'merge']].head(1).values[0])

n = pd.read_csv(data_directory + '/submission/mipacq_all_merge_metrics_entity_overlap_Procedures_1578676610.30274.csv')
ns = pd.read_csv(data_directory + '/submission/nested_with_singleton/mipacq_all_merge_metrics_entity_overlap_Procedures1578943851.142459.csv')
p = pd.read_csv(data_directory + '/submission/paired/mipacq_all_merge_metrics_entity_overlap_Procedures1578951619.047393.csv')

out = pd.concat([n, ns, p])

out.to_csv(data_directory + '/submission/combined/mipacq_Procedures.csv')

print('m proc', len(out), out.sort_values(by='F',ascending=False)[['F', 'merge']].head(1).values[0], out.sort_values(by='precision',ascending=False)[['precision', 'merge']].head(1).values[0], 
      out.sort_values(by='recall',ascending=False)[['recall', 'merge']].head(1).values[0])

n = pd.read_csv(data_directory + '/submission/mipacq_all_merge_metrics_entity_overlap_complete_1578672135.059832.csv')
ns = pd.read_csv(data_directory + '/submission/nested_with_singleton/mipacq_all_merge_metrics_entity_overlap_1578944504.010892.csv')
p = pd.read_csv(data_directory + '/submission/paired/mipacq_all_merge_metrics_entity_overlap_1578948214.410456.csv')

out = pd.concat([n, ns, p])

out.to_csv(data_directory + '/submission/combined/mipacq_all.csv')
print('m all', len(out), out.sort_values(by='F',ascending=False)[['F', 'merge']].head(1).values[0], out.sort_values(by='precision',ascending=False)[['precision', 'merge']].head(1).values[0], 
      out.sort_values(by='recall',ascending=False)[['recall', 'merge']].head(1).values[0])
'''            
                